In [11]:
# read data from csv file
import pandas as pd
import numpy as np
    
# read data from csv file
df = pd.read_csv('data.csv')

In [12]:
# Take a 10 sample of df
df_sample = df.sample(10)

In [13]:
#pip install geopy
from geopy.geocoders import Nominatim

# Get the coordinates of a city (Municipio)

def get_coordinates(city):

    geolocator = Nominatim(user_agent="spain_explorer")
    location = geolocator.geocode(city) 
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

# Create a new dataframe with the coordinates

df_sample['Latitude'], df_sample['Longitude'] = zip(*df_sample.apply(lambda x: get_coordinates(x['Provincia_FREQ']), axis=1))



In [14]:
# Create a column 'Num_Radiolinks' 

df_sample['Num_Radiolinks'] = 1

# Group the rows of the dataframe by the 'Municipio' column and sum the 'Num_Radiolinks' column
df_agg = df_sample.groupby(['Provincia_FREQ'])['Num_Radiolinks'].sum().reset_index()

# Rename the column 'Num_Radiolinks' to the desired name
df_agg.rename(columns={'Num_Radiolinks': 'Municipio_Count'}, inplace=True)

In [15]:
# add the 'Municipio_Count' from df_agg to the df_sample joining by 'Municipio'

df_sample = pd.merge(df_sample, df_agg, on='Provincia_FREQ')

In [47]:
import folium

# Create a map centered on Spain
spain_map = folium.Map(location=[40.416775, -3.703790], zoom_start=6)

# Iterate over the rows of the dataframe and add a marker for each municipality
for index, row in df_sample.iterrows():
    folium.Circle(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Municipality: {row['Provincia_FREQ']}<br>Number of Radiolinks: {row['Municipio_Count']}",
        radius=30000,
        fill=True,
        stroke=False,
        fill_color='blue',
        fillOpacity=0.2
    ).add_to(spain_map)

# Save the map as an HTML file
spain_map.save("spain_map_radiolinks.html")

In [48]:
# show the map

spain_map